In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [16]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    print(f'Carregando arquivo {file}')
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        loader = PyPDFLoader(file) #cada pagina vira um documento LangChain
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        loader = Docx2txtLoader(file)
    else:
        print('Formato não suportado!')
        return None

    data = loader.load()
    return data

# Wikipedia Loader
def load_from_wikipedia(query, lang='pt', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [27]:
# Teste carregando dados de um pdf
# documento = load_document('docs/CLT.pdf')
# data[100].metadata
# data[100].page_content
# len(data) #quantos documentos

In [28]:
# Teste carregando dados da wikipedia
# data = load_from_wikipedia('GPT-4')
# data[1].page_content

In [35]:
def chunk_data(data, chunk_size=1000):
    from langchain.text_splitter import RecursiveCharacterTextSplitter #splitter recomendado para texto genérico
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

def embedding_cost(texts: list) -> None:
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_tokens / 1000 * 0.0001:6f}')

In [36]:
data = load_document('docs/CLT.pdf')
chunks = chunk_data(data)

Carregando arquivo docs/CLT.pdf


In [37]:
chunks[10].page_content

'Título IV-A – Da Representação dos Empregados ........................................................................................ 88\nTítulo V – Da Organização Sindical ............................................................................................................... 89\nCapítulo I – Da Instituição Sindical ......................................................................................................... 89\nCapítulo II – Do Enquadramento Sindical ............................................................................................. 100\nCapítulo III – Da Contribuição Sindical ................................................................................................. 101\nTítulo VI – Das Convenções Coletivas de Trabalho .................................................................................... 109\nTítulo VI-A – Das Comissões de Conciliação Prévia ................................................................................... 113'

In [47]:
len(chunks) #quantos fragmentos/chunks/documentos (vector count)

861

In [44]:
embedding_cost(chunks)

Total de tokens: 252697
Custo de Embedding em USD: 0.025270


In [46]:
def insert_embeddings(index_name):
    import pinecone
    from langchain.vectorstores.pinecone import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name in pinecone.list_indexes():
        print(f'Index {index_name}')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('OK')
    else:
        print(f'Criando índice {index_name}')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('OK')
    return vector_store

In [41]:
def delete_index(index_name='all'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name == 'all':
        indexes = pinecone.list_indexes()
        print('Deletando todos os índices...')
        for index in indexes:
            pinecone.delete_index(index)
    else:
        print(f'Deletando índice {index_name}')
        pinecone.delete_index(index_name)

In [42]:
delete_index()

/opt/conda/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Deletando todos os índices...


In [45]:
index_name = 'linuxtips'
vector_store = insert_embeddings(index_name)

Criando index linuxtips
OK


In [53]:
def get_answer(vector_store, query):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(query)
    return answer


def ask_with_memory(vector_store, query, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(temperature=1) #default é model='gpt-3.5-turbo'
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': query, 'chat_history': chat_history})
    chat_history.append((query, result['answer']))

    return result, chat_history

In [49]:
query = 'depois de quanto tempo de trabalho tenho direito a férias?'
answer = get_answer(vector_store, query)
print(answer)

De acordo com o Decreto-Lei nº 1.535, de 13/4/1977, o empregado tem direito a férias após 12 meses de serviço.


In [51]:
import time
i = 1
print('Digite sair para encerrar.')
while True:
    query = input(f'Pergunta: #{i}: ')
    i = i+1
    if query.lower() in ['sair']:
        print('Encerrando')
        time.sleep(2)
        break

    answer = get_answer(vector_store, query)
    print(f'\nResposta: {answer}')
    print(f'\n {"-" * 50} \n')

Digite sair para encerrar.


Pergunta: #1:  o que é o décimo terceiro salário?



Resposta: O décimo terceiro salário é um benefício garantido aos trabalhadores no Brasil, que consiste no pagamento de um salário extra no final do ano. Ele é calculado com base no valor da remuneração do trabalhador e deve ser pago em duas parcelas: a primeira até o dia 30 de novembro e a segunda até o dia 20 de dezembro.

 -------------------------------------------------- 



Pergunta: #2:  qual foi a minha pergunta anterior?



Resposta: Desculpe, mas não consigo ver sua pergunta anterior. Você pode repeti-la?

 -------------------------------------------------- 



Pergunta: #3:  sair


Encerrando


In [54]:
chat_history = []
query = 'em que ano a CLT foi criada?'
result, chat_history = ask_with_memory(vector_store, query, chat_history)
print(result['answer'])

A CLT (Consolidação das Leis do Trabalho) foi criada em 1943.


In [55]:
query = 'estamos em 2023, quantos anos passaram?'
result, chat_history = ask_with_memory(vector_store, query, chat_history)
print(result['answer'])

Mais de setenta anos se passaram desde então.


In [59]:
print(chat_history)

[('em que ano a CLT foi criada?', 'A CLT (Consolidação das Leis do Trabalho) foi criada em 1943.'), ('estamos em 2023, quantos anos passaram?', 'Mais de setenta anos se passaram desde então.'), ('dê a quantidade exata de anos', 'Não é possível determinar a quantidade exata de anos que se passou desde então, pois o texto não especifica uma data específica para quando essas leis foram editadas.'), ('qual foi minha última pergunta?', 'Minha última pergunta foi "Qual é a sua pergunta?"')]
